In [1]:
! pip install flax
! pip install jaxtyping kagglehub treescope
! pip install ml_collections

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 2.0 MB/s eta 0:00:00


In [2]:
import jax
import jax.numpy as jnp
from flax import nnx
import optax
import os

In [3]:
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [4]:
from IPython.display import clear_output

VARIANT = 'paligemma-3b-pt-224' # @param ['paligemma-3b-mix-224', 'paligemma-3b-mix-448', 'paligemma-3b-pt-224', 'paligemma-3b-pt-448', 'paligemma-3b-pt-896'] {type:"string"}
paligemma_weights_dir = kagglehub.model_download(f'google/paligemma/JAX/{VARIANT}')
paligemma_weights_path = os.path.join(paligemma_weights_dir, f"{VARIANT}.npz")
print(f"Weights path: {paligemma_weights_path}")

Weights path: /kaggle/input/paligemma/jax/paligemma-3b-pt-224/1/paligemma-3b-pt-224.npz


In [5]:
tokenizer_path = "./paligemma_tokenizer.model"
if not os.path.exists(tokenizer_path):
  print("Downloading the model tokenizer...")
  !gsutil cp gs://big_vision/paligemma_tokenizer.model {tokenizer_path}
  print(f"Tokenizer path: {tokenizer_path}")

Copying gs://big_vision/paligemma_tokenizer.model...
- [1 files][  4.1 MiB/  4.1 MiB]                                                
Operation completed over 1 objects/4.1 MiB.                                      
Tokenizer path: ./paligemma_tokenizer.model


In [6]:
from flax import nnx
import sentencepiece as spm

In [7]:
paligemma_vocab = spm.SentencePieceProcessor()
paligemma_vocab.Load(tokenizer_path)

True

In [8]:
import sys
import tempfile
with tempfile.TemporaryDirectory() as tmp:
  # Create a temporary directory and clone the `flax` repo.
  # Then, append the `examples/gemma` folder to the path for loading the `gemma` modules.
  ! git clone https://github.com/Megan-Andrews/GSoC-2025.git {tmp}
  # ! ls {tmp}/src/models/gemma
  sys.path.append(f"{tmp}/src/models/paligemma/gemma")
  import sentencepiece as spm
  import params as params_lib
  import sampler as sampler_lib
  import transformer as transformer_lib
  import helpers as helpers
  import sow_lib
  sys.path.pop();

Cloning into '/tmp/tmpkjehrlwb'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 108 (delta 49), reused 88 (delta 31), pack-reused 0 (from 0)
Receiving objects: 100% (108/108), 1.97 MiB | 19.61 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [9]:
from flax import nnx
import sentencepiece as spm

In [10]:
paligemma_params = params_lib.load_and_format_params(paligemma_weights_path)

Loading parameters from /kaggle/input/paligemma/jax/paligemma-3b-pt-224/1/paligemma-3b-pt-224.npz...


In [11]:
sow_config = sow_lib.SowConfig(
    embeddings=True,
    rs_after_attention=True,
    rs_after_ffw=True,
    attn_logits_topk=5,
    mlp_hidden_topk=6,
)

In [12]:
transformer = transformer_lib.Transformer.from_params(paligemma_params, sow_config=sow_config)

In [13]:
sampler = sampler_lib.Sampler(
    transformer=transformer,
    vocab=paligemma_vocab,
)

In [14]:
input_batch = [
    "\n# Python program for implementation of Bubble Sort\n\ndef bubbleSort(arr):",
  ]

out_data = sampler(
    input_strings=input_batch,
    total_generation_steps=300,  # The number of steps performed when generating a response.
  )

for input_string, out_string in zip(input_batch, out_data.text):
  print(f"Prompt:\n{input_string}\nOutput:\n{out_string}")
  print()
  print(10*'#')

Prompt:

# Python program for implementation of Bubble Sort

def bubbleSort(arr):
Output:

for i in range(len(arr)):
for j in range(i+1):
if arr[j] > arr[j+1]:
swap(arr[j], arr[j+1])

##########


In [15]:
out_data.intermediates

TransformerIntermediates(embeddings=Array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 5.485346  ,  0.9305111 , -1.2922556 , ..., -0.27141246,
          0.0896271 , -0.25309056],
        [12.80112   , -1.9579765 , -9.2868595 , ..., -0.8405103 ,
          0.30166778, -1.6795954 ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]], dtype=float32), layers=LayerIntermediates(rs_after_attention=Array([[[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [-2.4770656 ,  2.0991364 ,  2.6474178 , ...,  0.5741457 ,
          -0.4195978 ,  0.7221151 ],
         [-1.1235981 ,  0.20481002, -1.5196404 , ..., -0.01349574,
 

# Minimal Cases for Debugging


In [ ]:
from __future__ import annotations
import functools
from transformer import _assign_linen_params_to_nnx_state, _map_linen_var_names
import modules
import layers
import flax.linen as nn

class Mini(nnx.Module):
  def __init__(
      self,
      *,
      rngs: nnx.Rngs,
  ):
    # Modified layers to be stacked.
    @nnx.split_rngs(splits=18)
    @nnx.vmap(in_axes=(None, 0), out_axes=0)
    def make_block(attn_type, rngs: nnx.Rngs):
        return modules.Block(
        num_heads=8,
        num_kv_heads=1,
        embed_dim=2048,
        head_dim=256,
        hidden_dim=256,
        sliding_window_size=None,
        use_post_attn_norm=False,
        use_post_ffw_norm=False,
        attn_logits_soft_cap=None,
        attn_type=1,
        query_pre_attn_scalar=256**-0.5,
        rngs=rngs,
        rope_base_frequency=10_000,
        rope_scale_factor=1.0,
        use_qk_norm=False,
        sow_config=sow_lib.SowConfig(embeddings=False, rs_after_attention=False, rs_after_ffw=False, mlp_hidden_topk=0, attn_logits_topk=0),
      )

    print("making_block")
    attn_type_ids = jnp.array([1 for _ in range(18)])
    self.layers = make_block(attn_type_ids, rngs)
    print("made_block")

  def __call__(
      self,
      last_tokens: Array,  # [B, L]
      positions: Array,  # [B, L]
      cache: Cache | None,  # (sequence length L')
      attention_mask: Array,  # [B, L, L']
  ) -> tuple[Array, Cache | None]:
    @nnx.split_rngs(splits=self.num_layers)
    @nnx.scan(in_axes=(nnx.Carry, 0), out_axes=(0, nnx.Carry))
    def forward_block(x, model: modules.Block):
      # TODO: pass cache
      x = model(x, positions, None , attention_mask)
      # TODO: set new_cache
      return x

    x = self.embedder.encode(last_tokens)
    _, x = forward_block(x, self.layers)

transformer = Mini(rngs=nnx.Rngs(params=0))


making_block
made_block


In [ ]:
import dataclasses
LayerCache = dict[str, jax.Array]
@dataclasses.dataclass(frozen=True)
class SowConfig():

  def sow_mlp(
      self,
      activations: jax.Array,
      module: nnx.Module,
  ):
    _, indices = jax.lax.top_k(jnp.abs(activations), 2)
    values = jnp.take_along_axis(activations, indices, axis=-1)
    module.sow(nnx.Intermediate, 'hidden_topk_values', values)
    module.sow(nnx.Intermediate, 'hidden_topk_indices', indices)



class Block(nnx.Module):
  def __init__(self, input_dim, features, sow_config, rngs):
    self.linear = nnx.Linear(input_dim, features, rngs=rngs)
    self.dropout = nnx.Dropout(0.5, rngs=rngs)
    self.sow_config = sow_config

  def __call__(self, x: jax.Array, cache: LayerCache):  # No need to require a second input!
    x = self.linear(x)
    self.sow_config.sow_mlp(x, self)
    x = self.dropout(x)
    x = jax.nn.relu(x)
    for k, v in cache.items():
      cache[k] = v + 1
    # print(cache)
    return x, cache   # No need to return a second output!

  def init_cache(
      self,
      cache_size: int,
      batch_size: int,
      dtype: jnp.dtype = jnp.bfloat16,
  ) -> LayerCache:
    return {
        'v': jnp.zeros(
            (batch_size, cache_size, 1, 1),
            dtype=dtype,
        ),
        'k': jnp.zeros(
            (batch_size, cache_size, 1, 1),
            dtype=dtype,
        ),
        'end_index': jnp.zeros((batch_size,), dtype=jnp.int32),
    }

class MLP(nnx.Module):
  def __init__(self, features, num_layers, rngs):
    self.sow_config = SowConfig()
    self.num_layers = num_layers

    @nnx.split_rngs(splits=num_layers)
    @nnx.vmap(in_axes=(0,), out_axes=0)
    def create_block(rngs: nnx.Rngs):
      return Block(features, features, self.sow_config, rngs=rngs)

    self.blocks = create_block(rngs)



  def __call__(self, x, cache: LayerCache):
    @nnx.split_rngs(splits=self.num_layers)
    @nnx.scan(in_axes=(nnx.Carry, 0, 0), out_axes=(nnx.Carry, 0))
    def forward(x, model, cache):
      x, new_cache = model(x, cache)
      return (x, new_cache)

    return forward(x, self.blocks, cache)

  def init_cache(self):

    @nnx.split_rngs(splits=self.num_layers)
    @nnx.scan(in_axes=0, out_axes=0)
    def scan_cache_init(block: Block):
        return block.init_cache(1, 1, jnp.bfloat16)  # replace with real args

    return scan_cache_init(self.blocks)


model = MLP(64, num_layers=5, rngs=nnx.Rngs(0))
# model(jnp.ones((3, 64)), {})
x = jnp.ones((3, 64))
cache = model.init_cache()
y, new_cache = model(x, cache)
y, new_cache = model(x, cache)
y, new_cache = model(x, new_cache)
model.blocks
print(model.blocks.linear.kernel.value.shape[0])
  # print(path)
  # print(model)
  # print()
# print(cache)
#

# print(new_cache)

# print(new_cache)

5


In [ ]:
print(getattr(model.blocks, 'hidden_topk_indices').value[0][:, :, 0, ...])
print("\n\n")
print(getattr(model.blocks, 'hidden_topk_indices').value[0])
print(getattr(model.blocks, 'hidden_topk_indices').value[0].shape)


[[ 6  6  6]
 [ 2 48 41]
 [50 56 17]
 [ 4 31 34]
 [16  4 23]]



[[[ 6 56]
  [ 6 56]
  [ 6 56]]

 [[ 2 42]
  [48 52]
  [41 30]]

 [[50 56]
  [56 24]
  [17 24]]

 [[ 4 15]
  [31 28]
  [34 55]]

 [[16 58]
  [ 4 36]
  [23 35]]]
(5, 3, 2)
